# Facility Location demo

Outline:
1. Data
2. Model
3. Analysis

Input:
Output:


# Data

In [ ]:
# import custom functions related to our model
import model
import data_preparation
# import libraries
import pandas as pd

In [ ]:
data_demand =pd.read_excel("C:/Data/Major city data.xlsx")


In [ ]:
# Show the data (input of the model)

# Model

# Analysis